In [1]:
import pandas as pd
import numpy as np

In [2]:
dataretrieve = pd.read_csv("..//datasets/dataretrieve.csv", na_values='?')
desharnais = pd.read_csv("..//datasets/desharnais.csv", na_values='?')
dataretrieve = dataretrieve.fillna(0)
desharnais = desharnais.fillna(0)

In [3]:
with open("..//datasets/dataretrieve.txt") as infile:
    dataretrievelist = infile.read()
    dataretrievelist = dataretrievelist.split(',')
with open("..//datasets/desharnais.txt") as infile:
    desharnaislist = infile.read()
    desharnaislist = desharnaislist.split(',')

In [26]:
from sklearn.preprocessing import StandardScaler

In [144]:
def myPCA(dataset, components):
    #A última coluna é a classe
    dataX = dataset.loc[:,dataset.columns[:-1]].values
    dataY = dataset.loc[:,[dataset.columns[-1]]].values
    
    dataXstd = StandardScaler().fit_transform(dataX)
    
    meanVec = np.mean(dataXstd, axis=0)
    covMat = np.cov(dataXstd.T)
    
    eigVals, eigVecs = np.linalg.eig(covMat)
    
    eigPairs = [(np.abs(eigVals[i]), eigVecs[:,i]) for i in range(len(eigVals))]
    
    totalInfo = sum(eigVals)
    infoCovered = [(i / totalInfo) for i in sorted(eigVals, reverse=True)]
    cumInfoCovered = np.cumsum(infoCovered)
    
    pos = 0
    if(components == 0):
        for i in range(len(cumInfoCovered)):
            if(i==len(cumInfoCovered)-1):
                pos = i-1
                break
            if (cumInfoCovered[i] > 0.9):
                pos = i
                break
    else:
        pos = components - 1
    
    matrixW = np.hstack((eigPairs[0][1].reshape(len(eigVals),1)))
    
    for j in range(1, pos+1):
        matrixW = np.c_[matrixW, eigPairs[j][1].reshape(len(eigVals),1)]
        
    transformedData = dataXstd.dot(matrixW)
    
    return transformedData, matrixW

In [158]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [213]:
def calcScores(dataset):
    knn = KNeighborsClassifier(n_neighbors=1)
    dataY = dataset.loc[:,[dataset.columns[-1]]].values.ravel()
    #c, r = dataY.shape
    #dataY = dataY.reshape(c,)
    scores = []
    pcs = []
    for i in range(1,len(dataset.columns)):
        transformedData, matrixW = myPCA(dataset, i)
        if(i==1):
            transformedData = transformedData.reshape((transformedData.shape[0],1))
        compScores = cross_val_score(knn, transformedData, dataY, cv=5, scoring='accuracy')
        scores.append(np.mean(compScores))
        pcs.append(i)
    return scores, pcs

In [214]:
dataretrieveScores, dataretrievePcs = calcScores(dataretrieve)

/Users/ajgan/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/ajgan/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/ajgan/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/ajgan/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/ajgan/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Da

In [215]:
dataretrieveScores

[0.86175498575498577,
 0.84547008547008551,
 0.86885470085470085,
 0.88423931623931618,
 0.87564672364672358,
 0.88423931623931618,
 0.88423931623931618,
 0.88423931623931618]

In [216]:
desharnaisScores, desharnaisPcs = calcScores(desharnais)

In [217]:
desharnaisScores

[0.44632352941176467,
 0.5286764705882353,
 0.54264705882352948,
 0.54264705882352948,
 0.49191176470588233,
 0.54191176470588232,
 0.57867647058823535,
 0.55367647058823533,
 0.57867647058823535,
 0.57867647058823535,
 0.57867647058823535]